

- From Mattman to another student on slack
    - Don't build the xml file, just use the docker
    - Run this: docker run -it -p 8764:8764 uscdatascience/inception-rest-tika
    - This is how to test it: http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://upload.wikimedia.org/wikipedia/commons/f/f6/Working_Dogs%2C_Handlers_Share_Special_Bond_DVIDS124942.jpg
    - more documentation here: https://cwiki.apache.org/confluence/display/TIKA/TikaAndVision#TikaAndVision-a.Usingdocker(Recommended)
    - to find image caption, use this docker: docker run -it -p 8764:8764 uscdatascience/im2txt-rest-tika
    - see this: https://github.com/USCDataScience/tika-dockers
    - then see this: http://localhost:8764/inception/v3/caption/image?url=https://upload.wikimedia.org/w[…]s_Thames_Tigers_2007.jpg/1200px-Marcus_Thames_Tigers_2007.jpg
    - additional documentation: https://cwiki.apache.org/confluence/display/TIKA/ImageCaption#ImageCaption-a.Usingdocker(Recommended)


# Follow these steps to get started:

Check out this if you get stuck: https://cwiki.apache.org/confluence/display/TIKA/TikaAndVision#TikaAndVision-a.Usingdocker(Recommended)

- install the docker desktop app
- pip install tensorflow
- git clone https://github.com/tensorflow/models.git
- export PYTHONPATH="$PYTHONPATH:`pwd`:`pwd`/slim"
- python -c 'import tensorflow, numpy, dataset; print("OK")'
- git clone https://github.com/USCDataScience/tika-dockers.git && cd tika-dockers
- docker build -f InceptionRestDockerfile -t uscdatascience/inception-rest-tika .
- docker run -p 8764:8764 -it uscdatascience/inception-rest-tika 
- Below is what Mattman said to do in slack. Note that the docker run command is very similar to the one we just did. I believe they are interchangable commands. 
- docker run -it -p 8764:8764 uscdatascience/inception-rest-tika
- to test it, plug this into web browser: http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://upload.wikimedia.org/wikipedia/commons/f/f6/Working_Dogs%2C_Handlers_Share_Special_Bond_DVIDS124942.jpg 
- if the test doesn't work, Go to docker desktop and clear out: 1) any unused or old containers; and 2) delete your image for uscdatascience/inception-rest-tika. Try again. 

In order to get a url that points to your downloaded photos, do the follwing:
- "python -mhttp.server 8888" in the directory where your photos are
- Follow this format to point to a specific photo http://localhost:8888/Pixstory-image-161668402049563.jpg
- here's another image to try: http://localhost:8888/Pixstory-image-162521751234631.jpeg
- if you want to see the list of links to images: http://localhost:8888



# Hypothesis - docker is not properly connected to local images. From Chat GPT: 
- First, you need to make sure that the image is stored in a directory that is accessible to the Docker container. You can do this by copying the image to the Docker host's file system, or by creating a shared volume between the host and the container.
    - docker run -it -p 8764:8764 -v /path/to/my/folder:/data uscdatascience/inception-rest-tika --debug
    - docker run -it -p 8764:8764 -v /Users/daniilabbruzzese/Documents/Senior Year/DSCI 550/assignment 2/DSCI550-PixstoryMediaExtractionAndAnalysis/4_Tika Image Dockers/95k_Images:/data uscdatascience/inception-rest-tika --debug


- Once the image is accessible to the Docker container, you can use the image's URL to access it from within the container. For example, if the image is stored at /path/to/image.jpg on the Docker host, you can access it from within the container using the URL http://host.docker.internal/path/to/image.jpg.

- When you run the Docker container, you need to make sure that the container is able to access the URL of the image. You can do this by specifying the appropriate network settings when you run the container. For example, you can use the --network host option to give the container access to the host network, or you can use the --add-host option to add the host's IP address to the container's /etc/hosts file.

- Once you have configured the network settings for the container, you should be able to access the image from within the container using its URL. You can test this by running the curl command from within the container, as we did earlier. If everything is set up correctly, you should be able to download the image from within the container.

# URL Pointing to Downloaded Images STEPS 

In order to get a url that points to your downloaded photos, do the follwing:
- "python -mhttp.server 8888" in the directory where your photos are
- Follow this format to point to a specific photo http://localhost:8888/Pixstory-image-161668402049563.jpg
- here's another image to try: http://localhost:8888/Pixstory-image-162521751234631.jpeg
- if you want to see the list of links to images: http://localhost:8888


ultimetly, you need the link to your photos to look like this: http://10.25.179.208:8888 AKA <your_local_ip>:8888.
Steps to do this: 

- Get IP address of your local machine: ipconfig (Windows) or ifconfig (Mac or Linux). 

- Configure the web server to bind to the host IP address: Once you have determined the IP address of your local machine, you can configure the web server to bind to that IP address. The steps to do this will depend on the web server software you are using, but typically involve editing a configuration file or settings file. Look for a setting that specifies the IP address or hostname that the web server should bind to, and set it to the IP address of your local machine. For example, in Apache, you would edit the httpd.conf file and set the Listen directive to Listen <your_local_ip>:8888.

- Configure Docker Desktop to access the web server: Once the web server is bound to the host IP address, you can configure Docker Desktop to access it. Open the Docker Desktop settings and go to the "Resources" section. Under "File Sharing", add the directory that contains your web server files, and make sure the checkbox next to it is selected. This will allow Docker Desktop to access the directory and the web server files.

- Test the web server in a Docker container: Once you have configured Docker Desktop to access the web server, you can test it by creating a Docker container that accesses the web server. You can do this by creating a Dockerfile that specifies the image you want to use, and the command to run the container (which would typically involve starting the web server). Then, build the image and run the container, and access the web server using the IP address and port number of the Docker container. For example, if your web server is running on http://localhost:8888 and your Docker container is running on http://172.17.0.2:8888, you would access the web server using http://172.17.0.2:8888.



In [9]:
import tika
import tika.parser
from tika import parser
import requests
import urllib.parse
import pandas as pd


In [11]:
#testing this with a pixtosry image online - it works! Even when the test link doesn't work.... I wonder why


# Load the CSV file into a pandas DataFrame
df = pd.read_csv('../Datasets/Master_Dataset_Raw.csv')


# Define a lambda function to modify the URLs
modify_url = lambda url: url.replace(".com/", ".com/optimized/")

# Apply the lambda function to the "Media" column of the DataFrame
df['Media'] = df['Media'].apply(modify_url)

first_media = df.loc[0, 'Media']
print(first_media)


#link is https://image.pixstory.com/optimized/Pixstory-image-165942680078373.jpeg
#modified link is 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://image.pixstory.com/optimized/Pixstory-image-165942680078373.jpeg'

url3 = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://image.pixstory.com/optimized/Pixstory-image-165942680078373.jpeg'

#another link is 

response = requests.get(url3)
json_response = response.json()

print(json_response)



https://image.pixstory.com/optimized/Pixstory-image-165942680078373.jpeg
{'confidence': [0.17712360620498657, 0.09326252341270447], 'classnames': ['loudspeaker, speaker, speaker unit, loudspeaker system, speaker system', 'face powder'], 'classids': [633, 552], 'time': {'read': 1370, 'units': 'ms', 'classification': 383}}


In [8]:
#Testing with downloaded image

#Example image URL "http://10.25.179.208:8888/Pixstory-image-164318304274781.png"

#Modified URL skeleton format 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=<URL>'


url3 = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=http://10.25.179.208:8888/Pixstory-image-164318304274781.png' #this one works!!!!!


response = requests.get(url3)
json_response = response.json()

print(json_response)


{'confidence': [0.675788402557373, 0.11660852283239365], 'classnames': ['pickelhaube', 'book jacket, dust cover, dust jacket, dust wrapper'], 'classids': [716, 922], 'time': {'read': 31, 'units': 'ms', 'classification': 423}}
